In [1]:
# Final Exam
import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.models as models

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Check if GPU is available
torch.cuda.is_available()
# If you see 'True', then you're good.

True

In [4]:
os.chdir('/content/drive/MyDrive/665Final')

In [5]:
class TestData(data.Dataset):
    def __init__(self, list_IDs, data_dir, transform):
        self.list_IDs = list_IDs
        self.data_dir = data_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.list_IDs)
    
    def __getitem__(self,index):
        ID = self.list_IDs[index]
        x = Image.open(os.path.join(self.data_dir,str(ID)+'.png'))      
        image = self.transform(x)
        return ID, image

In [ ]:
###################################################################
###################################################################
##################### Data Normalization ##########################
###################################################################
###################################################################
# def data_loaders(batch_size, data_dir, shuffle_test=False): 
#     trans_train = transforms.Compose([
#         transforms.ToTensor()
#         ])
#     trans_test = transforms.Compose([
#         transforms.ToTensor()
#         ])
#     trainset = datasets.ImageFolder(root = os.path.join(data_dir, 'train'), transform=trans_train)
#     train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#         shuffle=True, pin_memory=True)
#     return train_loader

# models_save_folder = './saved_models'
# os.makedirs(models_save_folder, exist_ok=True)
# batch_size = 100 
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# data_dir = 'final_exam_data/data' 
# train_loader= data_loaders(batch_size, data_dir)
# # Calculating Mean and STD of data
# mean = 0.
# std = 0.
# for images, _ in train_loader:
#     batch_samples = images.shape[0] 
#     images = images.view(batch_samples, images.shape[1], -1) 
#     mean += images.mean(2).sum(0) 
#     std += images.std(2).sum(0)
# mean /= len(train_loader.dataset)
# std /= len(train_loader.dataset)
# print(mean)
# print(std)



###################################################################

In [6]:
# Now we are ready to call our data loader
def data_loaders(batch_size, data_dir, shuffle_test=False): 
    normalize = transforms.Normalize(mean=[0.8260, 0.6706, 0.7601],
                                     std=[0.0977, 0.1722, 0.1293])#normalize
    trans_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
        ])
    trans_test = transforms.Compose([
        transforms.ToTensor(),
        normalize
        ])
    trainset = datasets.ImageFolder(root = os.path.join(data_dir, 'train'), transform=trans_train)
    valset = datasets.ImageFolder(root = os.path.join(data_dir, 'val'), transform=trans_test)
    testset = TestData(list(range(4800)),os.path.join(data_dir, 'test'), trans_test)
    
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
        shuffle=True, pin_memory=True)
    val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
        shuffle=shuffle_test, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
        shuffle=shuffle_test, pin_memory=True)
    class_to_idx = trainset.class_to_idx
    return train_loader, val_loader, test_loader, class_to_idx
models_save_folder = './saved_models'
os.makedirs(models_save_folder, exist_ok=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_dir = 'final_exam_data/data' #Path to the folder that contains the data
batch_size = 100
train_loader, val_loader, test_loader, class_to_idx = data_loaders(batch_size, data_dir)

In [7]:
# Experiment on pretrained models offered by torchvison.models
# My modifications include changing the dimension of the first and last layer
import torchvision.models as models
#vgg11
vgg11 = models.vgg11(pretrained=True)
vgg11.avgpool = nn.AdaptiveAvgPool2d((1, 1))
vgg11.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 3),
        )
#vgg11bn
vgg11bn = models.vgg11(pretrained=True)
vgg11bn.avgpool = nn.AdaptiveAvgPool2d((1, 1))
vgg11bn.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 3),
        )
bn = []
for layer in vgg11bn.features:
    bn += [layer]
    if isinstance(layer, nn.Conv2d):
        bn += [nn.BatchNorm2d(layer.out_channels)]
vgg11bn.features = nn.Sequential(*bn)
#vgg13
vgg13 = models.vgg13(pretrained=True)
vgg13.avgpool = nn.AdaptiveAvgPool2d((1, 1))
vgg13.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 3),
        )
#vgg13bn
vgg13bn = models.vgg13(pretrained=True)
vgg13bn.avgpool = nn.AdaptiveAvgPool2d((1, 1))
vgg13bn.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 3),
        )
bn = []
for layer in vgg13bn.features:
    bn += [layer]
    if isinstance(layer, nn.Conv2d):
      bn += [nn.BatchNorm2d(layer.out_channels)]
vgg13bn.features = nn.Sequential(*bn)
#resnet50
resnet50 = models.resnet50(pretrained=True)
resnet50.conv1 = nn.Conv2d(3, 64, kernel_size=(3,3), stride = (1,1), padding=(1,1))
resnet50.classifier = nn.Linear(2048,3)
resnet50.fc = nn.Linear(2048,3)
#resnet34
resnet34 = models.resnet34(pretrained=True)
resnet34.conv1 = nn.Conv2d(3, 64, kernel_size=(3,3), stride = (1,1), padding=(1,1))
resnet34.fc = nn.Linear(512,3)
resnet34.classifier = nn.Linear(512,3)
#densenet121
densenet121 = models.densenet121(pretrained=True)
densenet121.features.conv0 = nn.Conv2d(3, 64, kernel_size=(3,3), stride = (1,1), padding=(1,1))
densenet121.features.pool0 = nn.MaxPool2d(kernel_size=3, stride=2, padding = 1)
densenet121.classifier = nn.Linear(1024,3)
#DLA
from dla import DLA

Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/vgg13-19584684.pth" to /root/.cache/torch/hub/checkpoints/vgg13-19584684.pth


  0%|          | 0.00/508M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [8]:
class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB, nb_classes=3):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA #modelA : resnet50
        self.modelB = modelB #modelB : VGG11+bn
        # Remove last linear layer
        self.modelA.fc = nn.Identity()
        self.modelB.classifier[6] = nn.Identity()
        
        # Create new classifier
        self.classifier = nn.Linear(2048+4096, nb_classes)
        
    def forward(self, x):
        x1 = self.modelA(x.clone())  # clone to make sure x is not changed by inplace methods
        x1 = x1.view(x1.size(0), -1)
        x2 = self.modelB(x)
        x2 = x2.view(x2.size(0), -1)
        x = torch.cat((x1, x2), dim=1)
        
        x = self.classifier(nn.functional.relu(x))
        return x

In [9]:
def train_epoch(epoch, model_name, model, optimizer, data_loader, models_save_folder, training=True, save_model=False):
    if training:
        model = model.train().to(device)
    else:
        model = model.eval().to(device)     
        
    loss_fn = nn.CrossEntropyLoss() 

    time_start = time.time() 
    print('\nEpoch: %d' % epoch)
    loss = []
    correct = 0
    total = 0
    for i, (image, label) in tqdm(enumerate(data_loader)):
        if training: 
            optimizer.zero_grad() 
        image, label = image.to(device), label.to(device) 
        out = model(image) 
        batch_loss = loss_fn(out,label) 
        loss.append(batch_loss.item()) 

        if training:
            batch_loss.backward()
            optimizer.step()
        _, pred_label = torch.max(out.data, 1)
        total += image.shape[0]        
        correct += (pred_label == label).sum()

    time_elapsed = time.time() - time_start
    
    mean_epoch_loss = np.mean(loss)
    
    if save_model :
        save_path_cla = os.path.join(models_save_folder,
                             '{}_epoch{}_loss{:.2f}_acc{:.4f}.pt'.format(model_name, epoch, mean_epoch_loss, correct.item()/total))
        torch.save(model.state_dict(), save_path_cla)
    if training:
        print('\ndoing training for {}, Epoch {} took {:2f} s, loss={:.4f}, acc={:.4f}'.format(model_name, epoch, time_elapsed, 
                            mean_epoch_loss, correct.item()/total))
    else:
        print('\ndoing testing for{}, loss={:.4f}, acc={:.4f}'.format(model_name, mean_epoch_loss, correct.item()/total))
    return model, loss, correct.item()/total

In [10]:
def train_model(model_name, train_loader, test_loader,start_epoch, N_epochs, models_save_folder):
    if model_name.startswith('DLA'):
        model = DLA(num_classes=3)
    elif model_name=='vgg11':
      model = vgg11
    elif model_name=='vgg13':
      model = vgg13
    elif model_name=='vgg11bn':
      model = vgg11bn
    elif model_name=='vgg13bn':
      model = vgg13bn
    elif model_name.startswith('resnet34'):
     model = resnet34 
    elif model_name.startswith('resnet50'):
     model = resnet50
     
    elif model_name=='densenet121':
        model = densenet121
    elif model_name == 'ensemble_vggres':
      model_res = resnet50
      model_dir = 'saved_models/resnet50_epoch25_loss0.02_acc0.9923val0.9817.pt'
      state_dict = torch.load(model_dir) 
      model_res.load_state_dict(state_dict)

      model_vgg11 = vgg11bn
      model_dir = 'saved_models/vgg11bnlr001_epoch25_loss0.03_acc0.9902val0.9867.pt'
      state_dict = torch.load(model_dir) 
      model_vgg11.load_state_dict(state_dict)

      for param in model_res.parameters():
          param.requires_grad_(False)
      for param in model_vgg11.parameters():
          param.requires_grad_(False)
      model = MyEnsemble(model_res, model_vgg11)

    else:
        raise Exception("No such model option")   

    optimizer = optim.SGD(model.parameters(), lr=0.01,
                          momentum=0.9, weight_decay=5e-4) 
    
    val_acc=[]
    batch_loss=[]
    val_loss=[]

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
    for epoch in range(start_epoch, start_epoch + N_epochs): 
        _, loss1, _ =train_epoch(epoch, model_name , model , optimizer, train_loader, models_save_folder, training=True, save_model=True)
        _, loss2, val = train_epoch(epoch, model_name , model , optimizer, test_loader, models_save_folder, training=False, save_model=False)
        scheduler.step()
        val_acc.append(val)
        batch_loss.append(loss1)
        val_loss.append(loss2)
    return batch_loss, val_loss, val_acc

In [ ]:
# #For model comparison Experiment: run this code chunk by substituting 'modelA' for each model.
#modelA_bloss, modelA_valloss , modelA_valacc = train_model('modelA', train_loader, val_loader, 0, 30, models_save_folder)
#save_path_cla = os.path.join(models_save_folder, 'modelA.pt')
#torch.save({'modelA_bloss' : modelA_bloss, 'modelA_valloss' : modelA_valloss, 'modelA_valacc': modelA_valacc}, save_path_cla)

In [ ]:
esb_vggres_bloss, esb_vggres_valloss , esb_vggres_valacc = train_model('ensemble_vggres', train_loader, val_loader, 0, 20, models_save_folder)
save_path_cla = os.path.join(models_save_folder,
                             'esb_vggres.pt')
torch.save({'esb_vggres_bloss' : esb_vggres_bloss, 'esb_vggres_valloss' : esb_vggres_valloss, 'esb_vggres_valacc': esb_vggres_valacc}, save_path_cla)

In [16]:
# After training the model, use the generate_csv function to generate the prediction csv
idx_to_class = {v: k for k, v in class_to_idx.items()}

def id_to_class(label_num, idx_to_class):
    label_str = []
    for i in range(len(label_num)):
        label_str.append(idx_to_class[label_num[i].item()])
    return np.array(label_str)


def generate_csv(First_name, Last_name, model, test_loader, idx_to_class, vectorize=False):
    image_ids = []
    predicted_labels = []

    for i, (image_id, image) in tqdm(enumerate(test_loader)):
        if vectorize:
            image = image.view(image.shape[0],-1)
        image = image.to(device)
        out = model(image)
        _, pred_label = torch.max(out.data, 1)
        label_str = id_to_class(pred_label, idx_to_class)
        image_ids.append(image_id.numpy())
        predicted_labels.append(label_str)
    image_ids = np.concatenate(image_ids, 0)
    predicted_labels = np.concatenate(predicted_labels, 0)
    df = pd.DataFrame({'Image_ID':image_ids})
    df['Predicted_Labels'] = predicted_labels
    df.to_csv(os.path.join('./',First_name+Last_name+'.csv'))
    return

model = MyEnsemble(resnet50, vgg11bn)
model_dir = 'saved_models/ensemble_vggres_epoch8_loss0.01_acc0.9971val 0.9879.pt'
state_dict = torch.load(model_dir) 
model.load_state_dict(state_dict)
model = model.eval().to(device) 
generate_csv('Minji', 'Kim', model, test_loader, idx_to_class, vectorize=False)


48it [19:21, 24.19s/it]
